In [ ]:
# -*- coding: utf-8 -*-
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

EPISODES = 1000

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def greedy(self, state):
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        states, targets_f = [], []
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target 
            # Filtering out states and targets for training
            states.append(state[0])
            targets_f.append(target_f[0])
        #print(np.array(states).shape,np.array(targets_f).shape)
        #print(np.array(states), np.array(targets_f))
        #print("-----")
        history = self.model.fit(np.array(states), np.array(targets_f), epochs=1, verbose=0)
        # Keeping track of loss
        loss = history.history['loss'][0]
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return loss

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)



env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
# agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 32

for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            loss = agent.replay(batch_size)
            # Logging training loss every 10 timesteps
            if time % 10 == 0:
                print("episode: {}/{}, time: {}, loss: {:.4f}"
                    .format(e, EPISODES, time, loss))  
    if e % 10 == 0:
        agent.save("./save/cartpole-dqn.h5")


episode: 0/1000, score: 14, e: 1.0
episode: 1/1000, score: 15, e: 1.0
episode: 2/1000, score: 8, e: 0.97
episode: 3/1000, time: 0, loss: 4.8449
episode: 3/1000, time: 10, loss: 3.3651
episode: 3/1000, score: 11, e: 0.91
episode: 4/1000, time: 0, loss: 4.7022
episode: 4/1000, time: 10, loss: 2.1699
episode: 4/1000, score: 17, e: 0.84
episode: 5/1000, time: 0, loss: 4.2840
episode: 5/1000, time: 10, loss: 4.2074
episode: 5/1000, time: 20, loss: 3.1062
episode: 5/1000, score: 28, e: 0.73
episode: 6/1000, time: 0, loss: 1.9817
episode: 6/1000, time: 10, loss: 2.3675
episode: 6/1000, score: 11, e: 0.69
episode: 7/1000, time: 0, loss: 4.5398
episode: 7/1000, score: 10, e: 0.66
episode: 8/1000, time: 0, loss: 6.8521
episode: 8/1000, score: 8, e: 0.63
episode: 9/1000, time: 0, loss: 7.3477
episode: 9/1000, time: 10, loss: 9.5789
episode: 9/1000, score: 11, e: 0.6
episode: 10/1000, time: 0, loss: 2.8696
episode: 10/1000, score: 9, e: 0.57
episode: 11/1000, time: 0, loss: 4.7607
episode: 11/1000

In [2]:
# Play game
print("\nPlaying Game...")
sleep(1)

s = env.reset()
done = False
while not done:
    env.render()
    a = agent.greedy(np.array([s]))
    newS, r, done, _ = env.step(a)
    s = newS


Playing Game...


NameError: name 'sleep' is not defined